In [ ]:
import json
import pymongo
from pymongo import UpdateOne
import numpy
import math
from math import radians, cos, sin, asin, sqrt
from collections import Counter
import time
import sys
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.message import EmailMessage
import smtplib
import gc
gc.enable() 



def haversine(lng1, lat1, lng2, lat2):
    # 将十进制度数转化为弧度
    lng1, lat1, lng2, lat2 = map(radians, [lng1, lat1, lng2, lat2])
    # haversine公式
    dlng = lng2 - lng1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlng / 2) ** 2
    c = 2 * asin(sqrt(a))
    r = 6371  # 地球平均半径，单位为公里
    return c * r * 1000


b=time.time()
def mailTo(title,mailAdds,message,whoSend='taiwanInfoMaBoss'):
    msg = MIMEMultipart()    
    sender = whoSend
    subject = title
    body = message
    msg['From'] = sender
    msg['To'] = ','.join(mailAdds)
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))
    text=msg.as_string()
    #print text
    # Send the message via our SMTP server
    s = smtplib.SMTP('192.168.2.1',25)
    s.sendmail(sender,mailAdds, text)
    s.quit()  

mailTo(title="taiwanInfoMaBossbegin",mailAdds=["andy.yuan@wowprime.com"],message="分析開始")
try:
    ######################################################第一次跑用stoneTwo來篩選沒超商的點
    b=time.time()
    client = pymongo.mongo_client.MongoClient("192.168.1.113", 27017, username='j122085', password='850605')
    collectionTaiwan = client.rawData.taiwanInfoStoneTwo
#     collectionTaiwan = client.rawData.taiwanInfoHot7
    ##################################################第一次跑不執行
    TWLocs=list(collectionTaiwan.find({}, {'_id': False}))
    ##################################################第一次跑不執行
    
    ######################################################IPEEN
    # collection = client.rawData.wowprimeipeen
    collection = client.rawData.ipeenInfo
    ipeendata = list(collection.find({}, {'_id': False}))
    ipeendata = [dien for dien in ipeendata if dien['status'] == "正常營業"
                 and dien['lat'] > 18
                 and dien['lat'] < 27
                 and dien['lng'] < 125
                 and dien['lng'] > 117
                 and dien['bigadd'] != 0
                 and dien['smalladd'] != 0
                 and dien['averagecost'] < 250]

    ######################################################IPEEN
    ######################################################conStore0320
    collection = client.rawData.conStore
    conStoreData = list(collection.find({}, {'_id': False}))
    ######################################################conStore0320
    ######################################################Watsons
    collection = client.rawData.Watsons
    watsonsData = list(collection.find({}, {'_id': False}))
    ######################################################Watsons
    ######################################################carrefour0504
    collection = client.rawData.carrefour
    carrefourData = list(collection.find({}, {'_id': False}))
    ######################################################carrefour0504
    ######################################################pxmart0504
    collection = client.rawData.pxmart
    pxmartData = list(collection.find({}, {'_id': False}))
    ######################################################pxmart0504
    ######################################################Clinic0531
    collection = client.rawData.infoClinic
    clinicData = list(collection.find({}, {'_id': False}))
    ######################################################Clinic0531
    ######################################################Tstore0531
    collection = client.rawData.info3Store
    tStoreData = list(collection.find({}, {'_id': False}))
    ######################################################Tstore0531
    ##################################################第一次跑先去除latlng以外的資訊
    TWLocs = [{"lat":i["lat"],"lng":i["lng"]} for i in TWLocs if ('NconStore_Analyze' in i and
              i['NconStore_Analyze'] >0)]
#     TWLocs = [i for i in TWLocs if ('NconStore_Analyze' in i and
#               i['NconStore_Analyze'] >0)]
    ##################################################第一次跑先去除latlng以外的資訊
    allN=len(TWLocs)
    print("有{}筆資料要分析".format(allN))
    radius=500
    n = 0

    for TWLoc in TWLocs:
        score=0
        n += 1
        if n % 50 == 0:
            e=time.time()
            print("已完成{}%分析-{}項,花費{}秒".format(round(n/allN,4)*100,n,round(e-b)))
        if n % 2000 == 0:
            gc.collect()
            client = pymongo.mongo_client.MongoClient("192.168.1.113", 27017, username='j122085', password='850605')
            collection = client.rawData.taiwanInfoMaBoss
            
            ids=["{}_{}".format(i['lat'],i['lng']) for i in TWLocs if i.get("NconStore_Analyze",0)>0]
            datas=[i for i in TWLocs if i.get("NconStore_Analyze",0)>0]
            operations=[UpdateOne({"_id":idn},{'$set':data},upsert=True) for idn ,data in zip(ids,datas)]
            try:
                collection.bulk_write(operations ,ordered=False)
            except Exception as er:
                print(er)
            client.close()

        TWLoc["areaRadius_Analyze"] = radius
        #############################################################0523Score

        TWLoc["NconStore_Analyze"] = len([dien['name'] for dien in conStoreData if "lng" in dien and haversine(lng1=float(dien["lng"]),
                                                                                lat1=float(dien["lat"]),
                                                                                lng2=TWLoc["lng"],
                                                                                lat2=TWLoc["lat"]) <= radius])
            
        
        TWLoc["Nwatson_Analyze"] = len([dien for dien in watsonsData
                                          if haversine(lng1=float(dien["lng"]),
                                                       lat1=float(dien["lat"]),
                                                       lng2=TWLoc["lng"],
                                                       lat2=TWLoc["lat"]) <= radius])
        TWLoc["Npxmart_Analyze"] = len([dien for dien in pxmartData
                                          if haversine(lng1=float(dien["lng"]),
                                                       lat1=float(dien["lat"]),
                                                       lng2=TWLoc["lng"],
                                                       lat2=TWLoc["lat"]) <= radius])
        
            
#         TWLoc["NclinicData_Analyze"] = len([dien for dien in clinicData
#                                   if haversine(lng1=float(dien["lng"]),
#                                                lat1=float(dien["lat"]),
#                                                lng2=TWLoc["lng"],
#                                                lat2=TWLoc["lat"]) <= radius])
        
#         TWLoc["NtStore_Analyze"] = len([dien for dien in tStoreData
#                                   if haversine(lng1=float(dien["lng"]),
#                                                lat1=float(dien["lat"]),
#                                                lng2=TWLoc["lng"],
#                                                lat2=TWLoc["lat"]) <= radius])

        rangeCost = 1.4

        simDien = [dien['smallstyle'] for dien in ipeendata if haversine(lng1=dien["lng"],
                                                                       lat1=dien["lat"],
                                                                       lng2=TWLoc["lng"],
                                                                       lat2=TWLoc["lat"]) <= 500
                   and dien['averagecost'] < 200
                   and dien['averagecost'] > 160]
        
        TWLoc["NsimCostDien"] = len(simDien)

        threeMom = len([dien['smallstyle'] for dien in ipeendata if haversine(lng1=dien["lng"],
                                                                       lat1=dien["lat"],
                                                                       lng2=TWLoc["lng"],
                                                                       lat2=TWLoc["lat"]) <= 500
                   and "三媽" in dien['name']])
        
        TWLoc["NThreeMom_analyze"] = threeMom
        
        tea = len([dien['smallstyle'] for dien in ipeendata if haversine(lng1=dien["lng"],
                                                                       lat1=dien["lat"],
                                                                       lng2=TWLoc["lng"],
                                                                       lat2=TWLoc["lat"]) <= 500
                   and dien['smallstyle']=='飲料專賣店'])
        TWLoc["NTea_analyze"] = tea
        
        TWLoc["Nmc_Analyze"] = len([dien['name'] for dien in ipeendata 
                                if '麥當' in dien['name'] 
                                and haversine(lng1=dien["lng"],
                                              lat1=dien["lat"],
                                              lng2=TWLoc["lng"],
                                              lat2=TWLoc["lat"]) <= radius])
        TWLoc["Nken_Analyze"] = len([dien['name'] for dien in ipeendata 
                                if '肯德' in dien['name'] 
                                and haversine(lng1=dien["lng"],
                                              lat1=dien["lat"],
                                              lng2=TWLoc["lng"],
                                              lat2=TWLoc["lat"]) <= radius])

        #############################################################0523Score
#     for i in TWLocs:
#         score=0
#         if i['NclinicData_Analyze']>20:
#             score+=2
#         elif i['NclinicData_Analyze']>9:
#             score+=1
#         if i['NconStore_Analyze']>11:
#             score+=2
#         elif i['NconStore_Analyze']>5:
#             score+=1
#         if i['Npxmart_Analyze']+i['Nwatson_Analyze']>2:
#             score+=2
#         elif i['Npxmart_Analyze']+i['Nwatson_Analyze']>0:
#             score+=1

#         if i['NsimCostDien']>3:
#             score+=3
#         elif i['NsimCostDien']>2:
#             score+=2
#         elif i['NsimCostDien']>1:
#             score+=1
#         if i['NtStore_Analyze']+i["NDapu_analyze"]>0:
#             score+=1
#         i['score']=score
        
    
    e = time.time()


#     client.close()
    client = pymongo.mongo_client.MongoClient("192.168.1.113", 27017, username='j122085', password='850605')
    collection = client.rawData.taiwanInfoMaBoss
    ids=["{}_{}".format(i['lat'],i['lng']) for i in TWLocs]
    operations=[UpdateOne({"_id":idn},{'$set':data},upsert=True) for idn ,data in zip(ids,TWLocs)]
    try:
        collection.bulk_write(operations ,ordered=False)
    except Exception as er:
        print(er)
    client.close()
    
    e = time.time()
    print(e-b)
    
    
    mailTo(title="taiwanInfoMaBossSuccess",mailAdds=["andy.yuan@wowprime.com"],message="分析完畢_耗時{}秒".format(e-b))
except Exception as er:
    errorMsg=str(er)+"出錯位置:"+str(sys.exc_info()[2].tb_lineno)
    mailTo(title="taiwanInfoMaBossFail",mailAdds=["andy.yuan@wowprime.com"],message=errorMsg)

有50671筆資料要分析
已完成0.1%分析-50項,花費72秒
已完成0.2%分析-100項,花費131秒
已完成0.3%分析-150項,花費188秒
已完成0.38999999999999996%分析-200項,花費245秒
已完成0.49%分析-250項,花費307秒
已完成0.59%分析-300項,花費365秒
已完成0.69%分析-350項,花費425秒
已完成0.79%分析-400項,花費484秒
已完成0.89%分析-450項,花費543秒
已完成0.9900000000000001%分析-500項,花費601秒
已完成1.09%分析-550項,花費659秒
已完成1.18%分析-600項,花費717秒
已完成1.28%分析-650項,花費775秒
已完成1.38%分析-700項,花費833秒
已完成1.48%分析-750項,花費892秒
已完成1.58%分析-800項,花費956秒
已完成1.68%分析-850項,花費1023秒
已完成1.78%分析-900項,花費1083秒
已完成1.87%分析-950項,花費1143秒
已完成1.97%分析-1000項,花費1203秒
已完成2.07%分析-1050項,花費1261秒
已完成2.17%分析-1100項,花費1320秒
已完成2.27%分析-1150項,花費1381秒
已完成2.37%分析-1200項,花費1444秒
已完成2.4699999999999998%分析-1250項,花費1505秒
已完成2.5700000000000003%分析-1300項,花費1568秒
已完成2.6599999999999997%分析-1350項,花費1631秒
已完成2.76%分析-1400項,花費1693秒
已完成2.86%分析-1450項,花費1754秒
已完成2.96%分析-1500項,花費1814秒
已完成3.06%分析-1550項,花費1876秒
已完成3.16%分析-1600項,花費1935秒
已完成3.26%分析-1650項,花費1997秒
已完成3.35%分析-1700項,花費2056秒
已完成3.45%分析-1750項,花費2116秒
已完成3.55%分析-1800項,花費2179秒
已完成3.65%分析-1850項,花費2244秒
已完成3.75%分析-1900項,花費2307秒
已完成3.8

In [1]:
import pymongo
from pymongo import UpdateOne

client = pymongo.mongo_client.MongoClient("192.168.1.113", 27017, username='j122085', password='850605')
collection = client.rawData.ipeenInfo
ipeendata = list(collection.find({}, {'_id': False}))
ipeendata = [dien for dien in ipeendata if dien['status'] == "正常營業"
             and dien['lat'] > 18
             and dien['lat'] < 27
             and dien['lng'] < 125
             and dien['lng'] > 117
             and dien['bigadd'] != 0
             and dien['smalladd'] != 0
             and dien['averagecost'] < 250]

In [4]:
len([i for i in ipeendata if i['smallstyle']=='飲料專賣店'])#茶

4710

In [5]:
len([i for i in ipeendata if "三媽" in i['name']])

140

In [6]:
len([i for i in ipeendata if i['averagecost']<200 and i['averagecost']>160])

4062